### Chapter 2: Reading Time Series Data from Files 


In [2]:
import pandas as pd 
from pathlib import Path

#### Read data from a CSV file 

In [3]:
filepath =\
Path('../TimeSeriesAnalysisWithPythonCookbook/Data/movieboxoffice.csv')

In [4]:
ts = pd.read_csv(filepath, 
                 header=0,
                 parse_dates=['Date'],
                 index_col=0,
                 infer_datetime_format=True,
                 usecols=['Date', 'DOW', 'Daily', 'Forecast', 'Percent Diff'])
ts.head(5)

,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,"$125,789.89","$235,036.46",-46.48%
2021-04-27,Saturday,"$99,374.01","$197,622.55",-49.72%
2021-04-28,Sunday,"$82,203.16","$116,991.26",-29.74%
2021-04-29,Monday,"$33,530.26","$66,652.65",-49.69%
2021-04-30,Tuesday,"$30,105.24","$34,828.19",-13.56%


In [5]:
# Print a summary of the DataFrame
ts.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 128 entries, 2021-04-26 to 2021-08-31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DOW           128 non-null    object
 1   Daily         128 non-null    object
 2   Forecast      128 non-null    object
 3   Percent Diff  128 non-null    object
dtypes: object(4)
memory usage: 5.0+ KB


In [6]:
# Removing any non-numeric characters from the columns
clean = lambda x: x.str.replace('[^\d]', '', regex=True)
c_df = ts[['Daily', 'Forecast']].apply(clean, axis=1)
ts[['Daily', 'Forecast']] = c_df.astype(float)

In [7]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 128 entries, 2021-04-26 to 2021-08-31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DOW           128 non-null    object 
 1   Daily         128 non-null    float64
 2   Forecast      128 non-null    float64
 3   Percent Diff  128 non-null    object 
dtypes: float64(2), object(2)
memory usage: 5.0+ KB


In [8]:
# To get the exact memory usage for each column 
ts.memory_usage()

Index           1024
DOW             1024
Daily           1024
Forecast        1024
Percent Diff    1024
dtype: int64

In [9]:
# Total memory usage
ts.memory_usage().sum()

5120

There are situations where _parse_dates_ may not work! 
* in such cases the columns will be returned unchanged, and no error will be thrown 

This is where the _date_parser_ parameter can be useful! 

For example we can pass a lambda function that uses the _to_datetime_ function in pandas to _date_parser_. We can specify the string representation for the date forma inside _to_datetime()_

* %d represents the day of the month, such as 01 or 02 
* %b represents the abbreviated month name, such as Apr or May
* %y represents a two-digit year, such as 19 or 20 

Other common string codes include the following: 

* %Y represents the year as a for-digit number, such as 2020 or 2021
* %B represent the month's full name, such as January or February
* %m represents the month as a two-digit number such as 01 or 02

The _infer_datetime_format_ parameter in read_csv() function can speed up the parsing by 5-10x.

In [10]:
date_parser = lambda x: pd.to_datetime(x, format="%d-%b-%y")
ts = pd.read_csv(filepath, 
                 parse_dates=[0], 
                 index_col=0,
                 date_parser=date_parser,
                 usecols=[0,1,3,7,6])
ts.head()

,DOW,Daily,Forecast,Percent Diff
Date,,,,
2021-04-26,Friday,"$125,789.89","$235,036.46",-46.48%
2021-04-27,Saturday,"$99,374.01","$197,622.55",-49.72%
2021-04-28,Sunday,"$82,203.16","$116,991.26",-29.74%
2021-04-29,Monday,"$33,530.26","$66,652.65",-49.69%
2021-04-30,Tuesday,"$30,105.24","$34,828.19",-13.56%


In [11]:
ts = pd.read_csv(filepath,
                 header=0, 
                 parse_dates=[0], 
                 index_col=0,
                 infer_datetime_format=True, 
                 usecols=['Date', 'DOW', 'Daily', 'Forecast', 'Percent Diff'])

#### Reading data from an Excel File

In [12]:
filepath=\
Path('../TimeSeriesAnalysisWithPythonCookbook/Data/sales_trx_data.xlsx')


In [13]:
excelfile = pd.ExcelFile(filepath)

In [14]:
excelfile.sheet_names

['2017', '2018']

In [16]:
ts = pd.read_excel(filepath, 
                   engine='openpyxl',
                   index_col=1,
                   sheet_name=[0,1], 
                   parse_dates=True)
ts.keys()


dict_keys([0, 1])

In [17]:
# Reading from all the available sheets - we pass in None instead 

ts = pd.read_excel(filepath, 
                   engine='openpyxl',
                   index_col=1,
                   sheet_name=None, 
                   parse_dates=True)
ts.keys()


dict_keys(['2017', '2018'])

#### Multiple sheets

When we have multiple sheets with the same type of data, we can use the pd.concat() function to "union" the the data into one combined DF

In [19]:
# Combining the two DF's that are the result of the data of each of the sheets
ts_combined = pd.concat([ts['2017'], ts['2018']])
ts_combined.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 74124 entries, 2017-01-01 to 2018-12-31
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Line_Item_ID        74124 non-null  int64 
 1   Credit_Card_Number  74124 non-null  int64 
 2   Quantity            74124 non-null  int64 
 3   Menu_Item           74124 non-null  object
dtypes: int64(3), object(1)
memory usage: 2.8+ MB


In [20]:
# Multi-level index 
# To reduce the number of levels, we can use the droplevel(level=0) method to drop the first level af pandas .concat() 
ts_combined = pd.concat(ts).droplevel(level=0)

In [24]:
# Reading in only one specific sheet
ts = pd.read_excel(filepath, index_col=1, sheet_name='2018', parse_dates=True)
type(ts)

pandas.core.frame.DataFrame

We can also use the pd.ExcelFile() method to read in data from a spreadsheet!


In [26]:
excelFile = pd.ExcelFile(filepath)
excelFile.parse(sheet_name='2017', 
                index_col=1, 
                parse_dates=True).head()

,Line_Item_ID,Credit_Card_Number,Quantity,Menu_Item
Date,,,,
2017-01-01,1,7437926611570790,1,spicy miso ramen
2017-01-01,2,7437926611570790,1,spicy miso ramen
2017-01-01,3,8421920068932810,3,tori paitan ramen
2017-01-01,4,8421920068932810,3,tori paitan ramen
2017-01-01,5,4787310681569640,1,truffle butter ramen


In [44]:
#url = 'https://media.githubusercontent.com/media/PacktPublishing/Time-Series-Analysis-with-Python-Cookbook./main/datasets/Ch2/sales_trx_data.xlsx'
#url = 'https://github.com/PacktPublishing/Time-Series-Analysis-with-Python-Cookbook/blob/60fb737ee44eaeaa63f871b0cf4636aa11b0f6eb/datasets/Ch10/life_expectancy_birth.csv'
url = 'https://raw.githubusercontent.com/PacktPublishing/Time-Series-Analysis-with-Python-Cookbook/main/datasets/Ch2/AirQualityUCI.csv'

In [45]:
date_parser = lambda x: pd.to_datetime(x, format="%d/%m/%Y")

ts2 = pd.read_csv(url, 
                   delimiter=';',
                   date_parser=date_parser)


In [46]:
ts2

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18:00:00,2.6,1360.00,150,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.600,48.875001,0.757754
1,10/03/2004,19:00:00,2.0,1292.25,112,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.300,47.700000,0.725487
2,10/03/2004,20:00:00,2.2,1402.00,88,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.900,53.975000,0.750239
3,10/03/2004,21:00:00,2.2,1375.50,80,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.000,60.000000,0.786713
4,10/03/2004,22:00:00,1.6,1272.25,51,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.150,59.575001,0.788794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,04/04/2005,10:00:00,3.1,1314.25,-200,13.529605,1101.25,471.7,538.50,189.8,1374.25,1728.50,21.850,29.250000,0.756824
9353,04/04/2005,11:00:00,2.4,1162.50,-200,11.355157,1027.00,353.3,603.75,179.2,1263.50,1269.00,24.325,23.725000,0.711864
9354,04/04/2005,12:00:00,2.4,1142.00,-200,12.374538,1062.50,293.0,603.25,174.7,1240.75,1092.00,26.900,18.350000,0.640649
9355,04/04/2005,13:00:00,2.1,1002.50,-200,9.547187,960.50,234.5,701.50,155.7,1041.00,769.75,28.325,13.550000,0.513866
